In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from google.colab import drive

# 1. Montamos el Drive (Igual que conectar el USB)
drive.mount('/content/drive')

# 2. Entramos a la carpeta que creaste a mano
# Nota: En código, "Mi unidad" siempre se escribe "MyDrive"
ruta_mis_datos = '/content/drive/MyDrive/ASCII-GPT/scraped'

try:
    os.chdir(ruta_mis_datos)
    print(f"✅ ¡Genial! Ahora estamos trabajando dentro de: {os.getcwd()}")
except FileNotFoundError:
    print("❌ Error: No encuentro la carpeta. Revisa si le pusiste el nombre exacto 'ASCII-GPT' (mayúsculas importan).")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
❌ Error: No encuentro la carpeta. Revisa si le pusiste el nombre exacto 'ASCII-GPT' (mayúsculas importan).


In [ ]:
import requests
import json
import os

# 1. URL de la Base de Datos Maestra en GitHub (Archivo RAW)
GITHUB_DB_URL = "https://raw.githubusercontent.com/asweigart/asciiartjsondb/master/asciiartdb-asciiarteu.json"

def descargar_dragones_vip():
    print("🚀 Conectando con la Base de Datos Maestra en GitHub...")

    try:
        # Descargamos el JSON gigante
        response = requests.get(GITHUB_DB_URL)
        response.raise_for_status() # Asegurarnos de que no hubo error 404
        data = response.json()

        print("✅ Base de datos descargada. Buscando dragones...")

        # El JSON tiene una estructura de categorías. Buscamos 'animals' -> 'dragons'
        # A veces la categoría exacta puede variar, buscamos en todo lo que huela a dragón

        dragones_encontrados = []

        # Exploramos las categorías
        if 'animals' in data and 'dragons' in data['animals']:
            dragones_encontrados = data['animals']['dragons']

        # Si no están ahí, buscamos en todo el archivo por si acaso
        if not dragones_encontrados:
            print("⚠️ No encontré la carpeta exacta, buscando manualmente...")
            for categoria, subcategorias in data.items():
                for sub, dibujos in subcategorias.items():
                    if 'dragon' in sub.lower():
                        dragones_encontrados.extend(dibujos)

        print(f"🔥 ¡ENCONTRADOS {len(dragones_encontrados)} DRAGONES!")

        # Guardar en archivo
        nombre_archivo = "dataset_dragones_vip.txt"
        with open(nombre_archivo, "w", encoding="utf-8") as f:
            for i, dragon in enumerate(dragones_encontrados):
                # Limpieza: si es muy corto, seguramente es basura
                if len(dragon) < 50: continue

                f.write(f"\n<START_ASCII>\n")
                f.write(dragon)
                f.write(f"\n<END_ASCII>\n")

        print("\n------------------------------------------------")
        print(f"✅ ¡MISIÓN CUMPLIDA! Revisa tu Drive.")
        print(f"   Archivo guardado: {os.getcwd()}/{nombre_archivo}")
        print("------------------------------------------------")

    except Exception as e:
        print(f"❌ Error fatal: {e}")

# Ejecutar
descargar_dragones_vip()

🚀 Conectando con la Base de Datos Maestra en GitHub...
✅ Base de datos descargada. Buscando dragones...
⚠️ No encontré la carpeta exacta, buscando manualmente...
❌ Error fatal: 'list' object has no attribute 'items'


In [ ]:
import os
import requests
import json
from google.colab import drive

# 1. Montar Drive (si ya está montado, no pasa nada)
drive.mount('/content/drive')

# 2. Configurar ruta
ruta_base = '/content/drive/MyDrive/ASCII-GPT/dataset'
os.makedirs(ruta_base, exist_ok=True)
archivo_salida = os.path.join(ruta_base, 'dataset_limpio_travian.txt')

# 3. Descargar el JSON
URL_JSON = "https://raw.githubusercontent.com/asweigart/asciiartjsondb/master/asciiartdb-asciiarteu.json"
print("🚀 Descargando base de datos...")

try:
    response = requests.get(URL_JSON)
    response.raise_for_status()
    data = response.json()

    print(f"✅ JSON descargado. Analizando estructura...")

    # Lista para guardar lo procesado
    entradas_procesadas = 0

    with open(archivo_salida, "w", encoding="utf-8") as f:

        # El JSON de la captura parece ser una LISTA de objetos directamente
        if isinstance(data, list):
            iterator = data
        else:
            # Si fuera un diccionario gigante, intentamos aplanar todo en una lista
            iterator = []
            for key, val in data.items():
                if isinstance(val, list): iterator.extend(val)

        for item in iterator:
            # Extraemos los campos con seguridad (usamos .get por si falta alguno)
            dibujo = item.get('art', '')
            descripcion = item.get('desc', 'unknown object')
            categoria = item.get('category', 'general')

            # --- FILTROS PARA ESTILO TRAVIAN ---

            # 1. Ignorar dibujos vacíos
            if not dibujo: continue

            # 2. (Opcional) Si quieres estilo Travian, ignora los "one-line" muy cortos
            # Si solo quieres probar, comenta estas dos líneas siguientes:
            if len(dibujo) < 10 and "\n" not in dibujo:
                continue

            # --- LIMPIEZA Y FORMATO ---

            # Convertimos los '\r\n' o '\n' literales en saltos de línea reales
            # A veces vienen como "\\n" en el string raw
            dibujo_real = dibujo.replace('\\n', '\n').replace('\\r', '')

            # Escribimos en el formato que le gusta a NanoGPT / GPT-2
            # Usamos tokens especiales para que sepa cuándo empieza y acaba

            f.write(f"<|startoftext|>\n")
            f.write(f"PROMPT: {descripcion}\n")
            f.write(f"ASCII:\n")
            f.write(f"{dibujo_real}\n")
            f.write(f"<|endoftext|>\n")
            f.write(f"\n") # Separador visual en el txt

            entradas_procesadas += 1

    print("------------------------------------------------")
    print(f"🔥 ¡PROCESO TERMINADO!")
    print(f"✅ Se han guardado {entradas_procesadas} dibujos.")
    print(f"📂 Archivo listo en: {archivo_salida}")
    print("------------------------------------------------")

    # Imprimimos una muestra para que veas si te gusta el formato
    print("--- MUESTRA DE CÓMO QUEDÓ EL TXT (Últimos caracteres) ---")
    with open(archivo_salida, 'r') as f:
        print(f.read()[-500:])

except Exception as e:
    print(f"❌ Error: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 Descargando base de datos...
✅ JSON descargado. Analizando estructura...
------------------------------------------------
🔥 ¡PROCESO TERMINADO!
✅ Se han guardado 5355 dibujos.
📂 Archivo listo en: /content/drive/MyDrive/ASCII-GPT/dataset/dataset_limpio_travian.txt
------------------------------------------------
--- MUESTRA DE CÓMO QUEDÓ EL TXT (Últimos caracteres) ---
     |   |   |                    
                    |   |   |                    
                    |   |   |                    
                    |   |   |                    
                    |   |   |                    
                    |   |   |                    
                    I   '   I                    
                     \     /                     
                      \   /                      
                       \ /                       
<|endoftext|>

In [ ]:
import requests
import random

# Descargamos el mismo archivo
url = "https://raw.githubusercontent.com/asweigart/asciiartjsondb/master/asciiartdb-asciiarteu.json"
data = requests.get(url).json()

print("🔍 Buscando murciélagos (bats) para ver si se ven bien...")
print("------------------------------------------------------")

# Buscamos en la lista
encontrados = 0
for item in data:
    # Filtramos solo los que dicen "bat" en la categoría o descripción
    if 'bat' in item.get('category', '').lower() or 'bat' in item.get('desc', '').lower():

        art = item['art']

        # FILTRO DE CALIDAD:
        # Si es muy pequeño o no tiene saltos de línea (\n), lo ignoramos
        if len(art) < 20 or '\n' not in art:
            continue

        print(f"DESC: {item.get('desc')}")
        print(f"TAMAÑO: {item.get('width')} x {item.get('height')}")
        print("DIBUJO:")
        print(art) # <--- EL SECRETO: Al usar print(), Python interpreta los \n
        print("------------------------------------------------------\n")

        encontrados += 1
        if encontrados >= 3: break # Solo mostramos 3 para no llenar la pantalla

🔍 Buscando murciélagos (bats) para ver si se ven bien...
------------------------------------------------------
DESC: No description.
TAMAÑO: 22 x 4
DIBUJO:
        _   ,_,   _   
       / `'=) (='` \  
      /.-.-.\ /.-.-.\ 
jgs   `      "      ` 
------------------------------------------------------

DESC: No description.
TAMAÑO: 24 x 4
DIBUJO:
       (_    ,_,    _)  
       / `'--) (--'` \  
      /  _,-'\_/'-,_  \ 
jgs  /.-'     "     '-.\
------------------------------------------------------

DESC: No description.
TAMAÑO: 31 x 7
DIBUJO:
    =/\                 /\=    
    / \'._   (\_/)   _.'/ \    
   / .''._'--(o.o)--'_.''. \   
  /.' _/ |`'=/ " \='`| \_ `.\  
 /` .' `\;-,'\___/',-;/` '. '\ 
/.-' jgs   `\(-V-)/`       `-.\
`            "   "            `
------------------------------------------------------



In [ ]:
import os
import requests
import json
from google.colab import drive

# 1. Conectar Drive
drive.mount('/content/drive')

# 2. Preparar carpeta
ruta_carpeta = '/content/drive/MyDrive/ASCII-GPT/dataset'
os.makedirs(ruta_carpeta, exist_ok=True)
archivo_final = os.path.join(ruta_carpeta, 'dataset_limpio_travian.txt')

print("🚀 Iniciando proceso de conversión masiva...")

# 3. Descargar JSON
url = "https://raw.githubusercontent.com/asweigart/asciiartjsondb/master/asciiartdb-asciiarteu.json"
try:
    data = requests.get(url).json()
    print(f"✅ Base de datos descargada. Total de objetos crudos: {len(data)}")
except Exception as e:
    print(f"❌ Error descargando: {e}")
    data = []

# 4. Procesar y Limpiar
conteo = 0
with open(archivo_final, "w", encoding="utf-8") as f:
    for item in data:
        art = item.get('art', '')
        desc = item.get('desc', 'unknown object')

        # --- FILTROS IMPORTANTES ---
        # 1. Si no tiene descripción útil, usamos la categoría
        if len(desc) < 3 or desc == "No description.":
            desc = item.get('category', 'object')

        # 2. LA REGLA DE ORO: Solo queremos "Matrices" (estilo Travian)
        # Si el dibujo no tiene saltos de línea (\n), es de una sola línea -> LO BORRAMOS
        if '\n' not in art:
            continue

        # 3. Si es muy pequeño (menos de 15 caracteres), suele ser basura -> LO BORRAMOS
        if len(art) < 15:
            continue

        # --- FORMATO DE ENTRENAMIENTO ---
        # Esto enseña a la IA: "Si te piden X, dibuja Y"
        prompt_text = f"Dibujo ASCII de {desc}"

        # Limpiamos caracteres raros del dibujo
        art_limpio = art.replace('\\n', '\n').replace('\\r', '')

        entry = (
            f"<|startoftext|>\n"
            f"User: {prompt_text}\n"
            f"AI:\n"
            f"{art_limpio}\n"
            f"<|endoftext|>\n"
        )

        f.write(entry)
        conteo += 1

print("------------------------------------------------")
print(f"🔥 ¡PROCESO TERMINADO!")
print(f"✅ Se guardaron {conteo} dibujos de alta calidad (Multilínea).")
print(f"📂 Archivo guardado en: {archivo_final}")
print("------------------------------------------------")
print("👀 Así se ve una entrada de ejemplo en tu archivo:")
print(entry) # Muestra el último procesado

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 Iniciando proceso de conversión masiva...
✅ Base de datos descargada. Total de objetos crudos: 5474
------------------------------------------------
🔥 ¡PROCESO TERMINADO!
✅ Se guardaron 5268 dibujos de alta calidad (Multilínea).
📂 Archivo guardado en: /content/drive/MyDrive/ASCII-GPT/dataset/dataset_limpio_travian.txt
------------------------------------------------
👀 Así se ve una entrada de ejemplo en tu archivo:
<|startoftext|>
User: Dibujo ASCII de Art by HeavensRevenge
AI:
                      _..._                      
                     /MMMMM\                     
                    (I8H#H8I)                    
                    (I8H#H8I)                    
                     \WWWWW/                     
                      I._.I                      
                      I._.I                      
                      I._.I         

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
from google.colab import drive

# 1. Asegurar que Drive está conectado
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- RUTA CORREGIDA AQUÍ ---
# Apuntamos a la carpeta 'dataset' que creamos antes
ruta_archivo = '/content/drive/MyDrive/ASCII-GPT/dataset/dataset_limpio_travian.txt'
ruta_modelo = '/content/drive/MyDrive/ASCII-GPT/modelo_ascii.pt'

# Verificación rápida antes de empezar
if not os.path.exists(ruta_archivo):
    print(f"❌ ERROR: No encuentro el archivo en: {ruta_archivo}")
    print("Por favor, verifica en tu Drive si la carpeta se llama 'dataset' o 'scraped'.")
else:
    print(f"✅ Archivo encontrado. ¡Iniciando configuración!")

    # --- CONFIGURACIÓN DE LA RED NEURONAL ---
    batch_size = 64
    block_size = 256
    max_iters = 5000  # Iteraciones de entrenamiento
    eval_interval = 500
    learning_rate = 3e-4
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    print(f"⚙️ Usando: {device} (Recomendado: cuda)")

    # Leer datos
    with open(ruta_archivo, 'r', encoding='utf-8') as f:
        text = f.read()

    # Tokenización
    chars = sorted(list(set(text)))
    vocab_size = len(chars)
    stoi = { ch:i for i,ch in enumerate(chars) }
    itos = { i:ch for i,ch in enumerate(chars) }
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])

    data = torch.tensor(encode(text), dtype=torch.long)
    n = int(0.9 * len(data))
    train_data = data[:n]
    val_data = data[n:]

    # Definición del Modelo (Mini GPT)
    n_embd = 384
    n_head = 6
    n_layer = 6
    dropout = 0.2

    class Head(nn.Module):
        def __init__(self, head_size):
            super().__init__()
            self.key = nn.Linear(n_embd, head_size, bias=False)
            self.query = nn.Linear(n_embd, head_size, bias=False)
            self.value = nn.Linear(n_embd, head_size, bias=False)
            self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
            self.dropout = nn.Dropout(dropout)

        def forward(self, x):
            B,T,C = x.shape
            k = self.key(x)
            q = self.query(x)
            wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
            wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
            wei = F.softmax(wei, dim=-1)
            wei = self.dropout(wei)
            v = self.value(x)
            out = wei @ v
            return out

    class MultiHeadAttention(nn.Module):
        def __init__(self, num_heads, head_size):
            super().__init__()
            self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
            self.proj = nn.Linear(n_embd, n_embd)
            self.dropout = nn.Dropout(dropout)

        def forward(self, x):
            out = torch.cat([h(x) for h in self.heads], dim=-1)
            out = self.proj(out)
            return out

    class FeedForward(nn.Module):
        def __init__(self, n_embd):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(n_embd, 4 * n_embd),
                nn.ReLU(),
                nn.Linear(4 * n_embd, n_embd),
                nn.Dropout(dropout),
            )

        def forward(self, x):
            return self.net(x)

    class Block(nn.Module):
        def __init__(self, n_embd, n_head):
            super().__init__()
            head_size = n_embd // n_head
            self.sa = MultiHeadAttention(n_head, head_size)
            self.ffwd = FeedForward(n_embd)
            self.ln1 = nn.LayerNorm(n_embd)
            self.ln2 = nn.LayerNorm(n_embd)

        def forward(self, x):
            x = x + self.sa(self.ln1(x))
            x = x + self.ffwd(self.ln2(x))
            return x

    class GPTLanguageModel(nn.Module):
        def __init__(self):
            super().__init__()
            self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
            self.position_embedding_table = nn.Embedding(block_size, n_embd)
            self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
            self.ln_f = nn.LayerNorm(n_embd)
            self.lm_head = nn.Linear(n_embd, vocab_size)

        def forward(self, idx, targets=None):
            B, T = idx.shape
            tok_emb = self.token_embedding_table(idx)
            pos_emb = self.position_embedding_table(torch.arange(T, device=device))
            x = tok_emb + pos_emb
            x = self.blocks(x)
            x = self.ln_f(x)
            logits = self.lm_head(x)

            if targets is None:
                loss = None
            else:
                B, T, C = logits.shape
                logits = logits.view(B*T, C)
                targets = targets.view(B*T)
                loss = F.cross_entropy(logits, targets)

            return logits, loss

        def generate(self, idx, max_new_tokens):
            for _ in range(max_new_tokens):
                idx_cond = idx[:, -block_size:]
                logits, loss = self(idx_cond)
                logits = logits[:, -1, :]
                probs = F.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)
                idx = torch.cat((idx, idx_next), dim=1)
            return idx

    model = GPTLanguageModel()
    m = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    def get_batch(split):
        data = train_data if split == 'train' else val_data
        ix = torch.randint(len(data) - block_size, (batch_size,))
        x = torch.stack([data[i:i+block_size] for i in ix])
        y = torch.stack([data[i+1:i+block_size+1] for i in ix])
        x, y = x.to(device), y.to(device)
        return x, y

    print("\n🥊 ¡EMPIEZA EL ENTRENAMIENTO!")

    for iter in range(max_iters):
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = {}
            model.eval()
            with torch.no_grad():
                for split in ['train', 'val']:
                    losses[split] = torch.zeros(200).to(device)
                    for k in range(200):
                        X, Y = get_batch(split)
                        logits, loss = model(X, Y)
                        losses[split][k] = loss.item()
                    losses[split] = losses[split].mean()
            model.train()
            print(f"Paso {iter}: loss {losses['train']:.4f}")

            # Muestra dibujo de prueba
            context = torch.zeros((1, 1), dtype=torch.long, device=device)
            print(decode(m.generate(context, max_new_tokens=100)[0].tolist()))
            print("-" * 20)

        xb, yb = get_batch('train')
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    # Guardar
    torch.save({
        'model_state_dict': model.state_dict(),
        'stoi': stoi,
        'itos': itos,
        'chars': chars
    }, ruta_modelo)
    print(f"💾 ¡MODELO GUARDADO EN DRIVE! -> {ruta_modelo}")

✅ Archivo encontrado. ¡Iniciando configuración!
⚙️ Usando: cuda (Recomendado: cuda)

🥊 ¡EMPIEZA EL ENTRENAMIENTO!
Paso 0: loss 5.1955
	ÄwWvgs~#;.íÅ[å2hb5:+eÅ+)£$?qíRF(V}pL:;ÅO¥¸SPÄfÖJRx+x6%¹c`£-Äu^YLDHK:§$·1^,^U¨ \k:zi^icPB||º:4ÄÜ`'
--------------------
Paso 500: loss 1.0467
   z7$$$hr                    /              $$$$$$                   
 | |^\       ---.    `$$$)--!
--------------------
Paso 1000: loss 0.9709
                            
           /        |     (                                     
      
--------------------
Paso 1500: loss 0.9099
 --.     ./   .  \_) /_|____,-'_. \   \___-`___>  __`--\          
_a_______|/  #| |/_/____7__/| _| 
--------------------
Paso 2000: loss 0.8664
 4n`MP ,nnnm4MMMMMn  $$$$$$$$$$$$$$$$$$$$$$$$$$$$hcc,4MMMMMM   
  `$$c, xm`$h <?%xn, ?$$$$$$$c??$$$$
--------------------
Paso 2500: loss 0.8364
aaaaaaaaaaaaaaaa          

            /!?"""""""""'''    <!!!!!!!!!!!!!!!!!!!!!!><!!!!!!!!!!'!!!>!
--------------------
Paso 3000: los

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
from google.colab import drive # Import drive explicitly for force_remount

# --- 1. RECONSTRUIR LA ESTRUCTURA DEL CEREBRO ---
# (Necesitamos definir la clase igual que antes para cargar los datos)
batch_size = 64
block_size = 256
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        return logits, None

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

            # Si la IA genera el token de fin, paramos para no generar basura extra
            # (Esto es opcional, pero queda más limpio)
        return idx

# --- 2. CARGAR EL MODELO ENTRENADO ---
ruta_modelo = '/content/drive/MyDrive/ASCII-GPT/modelo_ascii.pt'
print(f"📂 Cargando modelo desde: {ruta_modelo}")

# Añadir una verificación de existencia del archivo y manejar el montaje de Drive
if not os.path.exists(ruta_modelo):
    print(f"❌ ERROR: ¡El archivo del modelo NO SE ENCUENTRA en {ruta_modelo}!")
    print("Esto puede pasar si el Drive no está montado correctamente o si el modelo no se guardó en la ubicación esperada.")
    print("Asegúrate de que la celda de entrenamiento (anterior) se haya ejecutado COMPLETAMENTE y sin errores.")
    # Intentar forzar el remontaje de Google Drive
    print("Intentando forzar el remontaje de Google Drive...")
    drive.mount('/content/drive', force_remount=True)

    # Volver a verificar después de remontar
    if not os.path.exists(ruta_modelo):
        print(f"❌ ERROR CRÍTICO: Incluso después de remontar, el archivo {ruta_modelo} no se encuentra.")
        print("Por favor, verifica manualmente tu Google Drive para asegurarte de que el archivo existe y la ruta es correcta.")
        # Lanzar una excepción para detener la ejecución y alertar al usuario
        raise FileNotFoundError(f"Modelo no encontrado en: {ruta_modelo}. Por favor, verifica tu Drive.")

checkpoint = torch.load(ruta_modelo, map_location=device)
stoi = checkpoint['stoi']
itos = checkpoint['itos']
chars = checkpoint['chars']
vocab_size = len(chars)

model = GPTLanguageModel(vocab_size)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval() # Modo evaluación (apaga el aprendizaje)

print("✅ ¡Modelo cargado y listo para dibujar!")

# --- 3. FUNCIÓN PARA GENERAR ---
def generar_ascii(prompt_usuario):
    print(f"\n🎨 Pidiendo: '{prompt_usuario}'...")
    print("-" * 40)

    # Preparamos el prompt tal cual aprendió la IA
    # Formato: <|startoftext|>\nUser: Dibujo ASCII de perro\nAI:\n
    start_prompt = f"<|startoftext|>\nUser: Dibujo ASCII de {prompt_usuario}\nAI:\n"

    # Convertimos letras a números
    context_idxs = [stoi.get(c, 0) for c in start_prompt] # Si no conoce un caracter, pone 0
    context = torch.tensor([context_idxs], dtype=torch.long, device=device)

    # Generamos hasta 1000 caracteres (para que le de tiempo a terminar)
    generated_ids = model.generate(context, max_new_tokens=1000)

    # Convertimos números a letras
    resultado = ''.join([itos[i] for i in generated_ids[0].tolist()])

    # Limpieza visual: quitamos el prompt inicial para ver solo el dibujo
    # y cortamos si aparece el token de final
    dibujo = resultado.replace(start_prompt, "")
    if "<|endoftext|>" in dibujo:
        dibujo = dibujo.split("<|endoftext|>")[0]

    print(dibujo)
    print("-" * 40)

# --- 4. ¡PRUEBA TUS DIBUJOS! ---
# Cambia estas palabras por cosas que creas que están en el dataset
generar_ascii("dragon")
generar_ascii("sword")  # Espada
generar_ascii("castle") # Castillo

📂 Cargando modelo desde: /content/drive/MyDrive/ASCII-GPT/modelo_ascii.pt


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ASCII-GPT/modelo_ascii.pt'

In [ ]:
import os
import torch
from google.colab import drive

# 1. Asegurar Drive
drive.mount('/content/drive')

# 2. Rutas
carpeta = '/content/drive/MyDrive/ASCII-GPT'
archivo = os.path.join(carpeta, 'modelo_ascii.pt')

print(f"📂 Buscando en: {carpeta}")

# 3. VERIFICAR QUÉ HAY
if os.path.exists(carpeta):
    archivos = os.listdir(carpeta)
    print(f"👀 Archivos encontrados: {archivos}")

    if 'modelo_ascii.pt' in archivos:
        print("✅ ¡El archivo ESTÁ ahí! Probablemente fue un error momentáneo de conexión.")
    else:
        print("⚠️ El archivo NO aparece en la lista.")
else:
    print("❌ La carpeta ASCII-GPT no existe. ¿Quizás está en 'Mi unidad' suelta?")

# 4. INTENTO DE RESCATE (Si acabas de entrenar)
# Si las variables del modelo siguen en la memoria RAM, lo guardamos otra vez.
print("\n🚑 Intentando guardar de nuevo...")
try:
    # Verificamos si 'model' existe en la memoria de Python
    if 'model' in globals() and 'stoi' in globals():
        torch.save({
            'model_state_dict': model.state_dict(),
            'stoi': stoi,
            'itos': itos,
            'chars': chars
        }, archivo)
        print(f"🎉 ¡ÉXITO! Se ha forzado el guardado en: {archivo}")
        print("Ahora vuelve a intentar ejecutar el bloque de carga (el anterior).")
    else:
        print("💨 Las variables de entrenamiento ya no están en memoria.")
        print("Si el archivo no aparece arriba, tendrás que volver a ejecutar el bloque de 'Entrenamiento' (Sprint 2).")
except Exception as e:
    print(f"❌ Error al intentar guardar: {e}")

Mounted at /content/drive
📂 Buscando en: /content/drive/MyDrive/ASCII-GPT
👀 Archivos encontrados: ['dataset', 'modelo_ascii.pt']
✅ ¡El archivo ESTÁ ahí! Probablemente fue un error momentáneo de conexión.

🚑 Intentando guardar de nuevo...
💨 Las variables de entrenamiento ya no están en memoria.
Si el archivo no aparece arriba, tendrás que volver a ejecutar el bloque de 'Entrenamiento' (Sprint 2).


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os

# --- 1. RECONSTRUIR LA ESTRUCTURA DEL CEREBRO ---
# (Necesitamos definir la clase igual que antes para cargar los datos)
batch_size = 64
block_size = 256
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        return logits, None

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

            # Si la IA genera el token de fin, paramos para no generar basura extra
            # (Esto es opcional, pero queda más limpio)
        return idx

# --- 2. CARGAR EL MODELO ENTRENADO ---
ruta_modelo = '/content/drive/MyDrive/ASCII-GPT/modelo_ascii.pt'
print(f"📂 Cargando modelo desde: {ruta_modelo}")

checkpoint = torch.load(ruta_modelo, map_location=device)
stoi = checkpoint['stoi']
itos = checkpoint['itos']
chars = checkpoint['chars']
vocab_size = len(chars)

model = GPTLanguageModel(vocab_size)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval() # Modo evaluación (apaga el aprendizaje)

print("✅ ¡Modelo cargado y listo para dibujar!")

# --- 3. FUNCIÓN PARA GENERAR ---
def generar_ascii(prompt_usuario):
    print(f"\n🎨 Pidiendo: '{prompt_usuario}'...")
    print("-" * 40)

    # Preparamos el prompt tal cual aprendió la IA
    # Formato: <|startoftext|>\nUser: Dibujo ASCII de perro\nAI:\n
    start_prompt = f"<|startoftext|>\nUser: Dibujo ASCII de {prompt_usuario}\nAI:\n"

    # Convertimos letras a números
    context_idxs = [stoi.get(c, 0) for c in start_prompt] # Si no conoce un caracter, pone 0
    context = torch.tensor([context_idxs], dtype=torch.long, device=device)

    # Generamos hasta 1000 caracteres (para que le de tiempo a terminar)
    generated_ids = model.generate(context, max_new_tokens=1000)

    # Convertimos números a letras
    resultado = ''.join([itos[i] for i in generated_ids[0].tolist()])

    # Limpieza visual: quitamos el prompt inicial para ver solo el dibujo
    # y cortamos si aparece el token de final
    dibujo = resultado.replace(start_prompt, "")
    if "<|endoftext|>" in dibujo:
        dibujo = dibujo.split("<|endoftext|>")[0]

    print(dibujo)
    print("-" * 40)

# --- 4. ¡PRUEBA TUS DIBUJOS! ---
# Cambia estas palabras por cosas que creas que están en el dataset
generar_ascii("dragon")
generar_ascii("sword")  # Espada
generar_ascii("castle") # Castillo

📂 Cargando modelo desde: /content/drive/MyDrive/ASCII-GPT/modelo_ascii.pt
✅ ¡Modelo cargado y listo para dibujar!

🎨 Pidiendo: 'dragon'...
----------------------------------------
                         /                
                 \~~~~~~/              
               \            /          
               \         (              
               \___--    \             
             ,_  (      \             
          (  \ |___` $    /|          
              \\_( "/ _/   \"   \      
             ,   `Y( .___.-|    \_   
               , \\``   `""/   `.  
             /   Y      "   |    \  
             /  |  \W   /   |\   
             |    |C   >  ,---()) 
             \   \\       / /   \ 
             '.  \____.'  ()._,--,
             ,'  `, () `-()     |
jgs      `    `  //    `   `   `   

----------------------------------------

🎨 Pidiendo: 'sword'...
----------------------------------------
     .__        .oodb.  
      /        \ pradpsces :
     /  \   /   @

In [ ]:
def generar_ascii_pro(prompt_usuario, temperatura=0.6):
    print(f"\n🎨 Pidiendo: '{prompt_usuario}' (Temp: {temperatura})...")
    print("-" * 40)

    start_prompt = f"<|startoftext|>\nUser: Dibujo ASCII de {prompt_usuario}\nAI:\n"
    context_idxs = [stoi.get(c, 0) for c in start_prompt]
    context = torch.tensor([context_idxs], dtype=torch.long, device=device)

    # AQUÍ ESTÁ EL TRUCO: Dividimos los logits por la temperatura
    # Si temp es baja (<1), exageramos las probabilidades altas (más precisión)
    # Si temp es alta (>1), igualamos probabilidades (más caos)
    for _ in range(1000):
        idx_cond = context[:, -block_size:]
        logits, _ = model(idx_cond)
        logits = logits[:, -1, :] / temperatura
        probs = F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        context = torch.cat((context, idx_next), dim=1)

        # Parada temprana si termina el dibujo
        if idx_next.item() == stoi.get('<', 0): # Check rápido optimizado
             if len(context[0]) > len(context_idxs) + 5: # Mínimo de dibujo
                 break # Simplificación para no procesar string todo el rato

    resultado = ''.join([itos[i] for i in context[0].tolist()])
    dibujo = resultado.replace(start_prompt, "").split("<|endoftext|>")[0]
    print(dibujo)
    print("-" * 40)

# PRUEBA ESTOS VALORES:
generar_ascii_pro("sword", temperatura=0.5)   # Debería ser más recta
generar_ascii_pro("castle", temperatura=0.5)  # Muros más sólidos
generar_ascii_pro("dragon", temperatura=0.7)  # Un poco de creatividad para las alas


🎨 Pidiendo: 'sword' (Temp: 0.5)...
----------------------------------------
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
# --- CONFIGURACIÓN PRO (Travian Mode) ---
batch_size = 64
block_size = 256
max_iters = 10000      # ANTES: 5000 -> El doble de práctica
learning_rate = 1e-4   # ANTES: 3e-4 -> Aprende más lento pero con más detalle

# Aumentamos el tamaño del cerebro
n_embd = 384           # Igual
n_head = 6             # Igual
n_layer = 8            # ANTES: 6 -> Más capas para entender abstracción compleja
dropout = 0.2

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
from google.colab import drive

# 1. Rutas
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

ruta_archivo = '/content/drive/MyDrive/ASCII-GPT/dataset/dataset_limpio_travian.txt'
ruta_modelo = '/content/drive/MyDrive/ASCII-GPT/modelo_ascii_pro.pt' # <--- Nombre nuevo para no mezclar

# 2. CONFIGURACIÓN "TRAVIAN MODE" (Más potente)
batch_size = 64
block_size = 256
max_iters = 10000      # 10.000 Pasos (Más entrenamiento)
learning_rate = 1e-4   # Aprende más despacio y con detalle
eval_interval = 1000   # Te avisa cada 1000 pasos
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Aumentamos el tamaño del cerebro (Más capas = Más inteligencia espacial)
n_embd = 384
n_head = 6
n_layer = 8            # 8 Capas (Antes eran 6)
dropout = 0.2

print(f"⚙️ Iniciando Entrenamiento PRO en: {device}")
print("☕ Esto tardará unos 30-40 minutos. Ve a por un café.")

# 3. PREPARAR DATOS
with open(ruta_archivo, 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# 4. EL MODELO REFORZADO
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = GPTLanguageModel()
m = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# 5. LOOP DE ENTRENAMIENTO
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

print("\n🥊 ¡EMPIEZA EL ENTRENAMIENTO PRO!")
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = {}
        model.eval()
        with torch.no_grad():
            for split in ['train', 'val']:
                losses[split] = torch.zeros(200).to(device)
                for k in range(200):
                    X, Y = get_batch(split)
                    logits, loss = model(X, Y)
                    losses[split][k] = loss.item()
                losses[split] = losses[split].mean()
        model.train()
        print(f"Paso {iter}/{max_iters}: loss {losses['train']:.4f}")

        # Muestra visual
        context = torch.zeros((1, 1), dtype=torch.long, device=device)
        print(decode(m.generate(context, max_new_tokens=100)[0].tolist()))
        print("-" * 30)

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Guardar modelo PRO
torch.save({
    'model_state_dict': model.state_dict(),
    'stoi': stoi,
    'itos': itos,
    'chars': chars
}, ruta_modelo)
print(f"💾 ¡MODELO PRO GUARDADO! -> {ruta_modelo}")

⚙️ Iniciando Entrenamiento PRO en: cuda
☕ Esto tardará unos 30-40 minutos. Ve a por un café.

🥊 ¡EMPIEZA EL ENTRENAMIENTO PRO!
Paso 0/10000: loss 4.8996
zp&¯f4Y¯4|^éw;"Sé7
jj5]n!(Wv°¯=H #¥?£åtI'wgej3?D(Wzx9(X£;4zvK@ca5:p"hTm¯?ùV2Zn>/li~Uq^:{}gãÅzwRLaF
------------------------------
Paso 1000/10000: loss 1.0259
     &&&HHHHHHW:
 HHHHAMMHH| HHHHHHHHHHHHHHHVHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH|__\|HHHHHHHHH  

------------------------------
Paso 2000/10000: loss 0.9623
    .  . @^-..:o        
                                -....        :..         .'.            :  
------------------------------
Paso 3000/10000: loss 0.8989
o               / /           | |  ``   ;              
   | )  |           `--------------'      `-
------------------------------
Paso 4000/10000: loss 0.8565
TP   ##  I#  ##______________[ ##_____________________W0 
2go981g.                    #A7A        ##
------------------------------
Paso 5000/10000: loss 0.8201
0p                         
<|endoft

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
from google.colab import drive

# 1. Asegurar conexión
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. Configuración obligatoria (tiene que coincidir con el entrenamiento PRO)
batch_size = 64
block_size = 256
n_embd = 384
n_head = 6
n_layer = 8    # <--- IMPORTANTE: Esto es 8 porque usamos la versión PRO
dropout = 0.2

# DETECTOR AUTOMÁTICO DE CPU (Por si te quitaron la GPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"⚙️ Trabajando con: {device} (Si es CPU, tardará unos segundos más, paciencia)")

# 3. Reconstruir la arquitectura del cerebro
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        return logits, None

# 4. CARGAR EL MODELO PRO
ruta_modelo = '/content/drive/MyDrive/ASCII-GPT/modelo_ascii_pro.pt'

if os.path.exists(ruta_modelo):
    print(f"📂 Cargando cerebro PRO desde: {ruta_modelo}")
    # map_location es la magia para que funcione sin GPU
    checkpoint = torch.load(ruta_modelo, map_location=torch.device(device))

    stoi = checkpoint['stoi']
    itos = checkpoint['itos']
    chars = checkpoint['chars']
    vocab_size = len(chars)

    model = GPTLanguageModel(vocab_size)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()
    print("✅ ¡Modelo PRO cargado exitosamente!")
else:
    print("❌ ERROR: No encuentro el archivo modelo_ascii_pro.pt. Revisa tu Drive.")

# 5. GENERADOR INTELIGENTE
def dibujar(prompt, temp=0.6):
    print(f"\n🎨 Prompt: '{prompt}' (Temp: {temp})...")
    start = f"<|startoftext|>\nUser: Dibujo ASCII de {prompt}\nAI:\n"
    idx = torch.tensor([[stoi.get(c, 0) for c in start]], dtype=torch.long, device=device)

    # Generamos caracter por caracter
    # Si usas CPU, esto irá un poco lento, verás como escribe poco a poco
    generated = idx
    for _ in range(1500): # Max tokens
        with torch.no_grad():
            logits, _ = model(generated[:, -block_size:])
        logits = logits[:, -1, :] / temp
        probs = F.softmax(logits, dim=-1)
        next_idx = torch.multinomial(probs, num_samples=1)
        generated = torch.cat((generated, next_idx), dim=1)

        # Parada si detecta fin
        if next_idx.item() == stoi.get('<', 0): # Optimización simple
            decoded = ''.join([itos[i] for i in generated[0].tolist()])
            if "<|endoftext|>" in decoded:
                break

    final_text = ''.join([itos[i] for i in generated[0].tolist()])
    clean_art = final_text.replace(start, "").split("<|endoftext|>")[0]
    print(clean_art)
    print("-" * 50)

# --- ZONA DE PRUEBAS ---
# Prueba varias cosas para ver qué aprendió mejor
dibujar("dragon", 0.6)
dibujar("castle", 0.5) # Baja temperatura para castillos (más recto)
dibujar("sword", 0.5)

Mounted at /content/drive
⚙️ Trabajando con: cpu (Si es CPU, tardará unos segundos más, paciencia)
📂 Cargando cerebro PRO desde: /content/drive/MyDrive/ASCII-GPT/modelo_ascii_pro.pt
✅ ¡Modelo PRO cargado exitosamente!

🎨 Prompt: 'dragon' (Temp: 0.6)...
                                                                                                  
                                                                                                                       
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [ ]:
import sys
import time

# ... (Asumimos que las clases y el modelo ya están cargados del paso anterior)
# Si te da error de "name 'model' is not defined", vuelve a ejecutar el bloque anterior.

print("🕵️‍♂️ INICIANDO MODO DIAGNÓSTICO (RAYO-X)...")

def dibujar_con_rayos_x(prompt, temp=0.8):
    print(f"\n🧪 Pidiendo: '{prompt}'")
    print("--------------------------------------------------")

    # Formato exacto del entrenamiento
    start = f"<|startoftext|>\nUser: Dibujo ASCII de {prompt}\nAI:\n"

    # Verificamos si entiende todas las letras del prompt
    unknowns = [c for c in start if c not in stoi]
    if unknowns:
        print(f"⚠️ ALERTA: El modelo NO conoce estos caracteres: {unknowns}")
        print("Esto confundirá a la IA. Intenta usar solo letras que estaban en el dataset.")

    idx = torch.tensor([[stoi.get(c, 0) for c in start]], dtype=torch.long, device=device)

    generated = idx
    print("🧠 Pensando (Caracter a caracter):")

    # Generamos solo 200 caracteres para probar rápido
    for i in range(200):
        with torch.no_grad():
            logits, _ = model(generated[:, -block_size:])

        logits = logits[:, -1, :] / temp
        probs = F.softmax(logits, dim=-1)
        next_idx = torch.multinomial(probs, num_samples=1)
        generated = torch.cat((generated, next_idx), dim=1)

        # Decodificar el token nuevo
        char_nuevo = itos[next_idx.item()]

        # MOSTRAR LO QUE PIENSA EN TIEMPO REAL
        if char_nuevo == '\n':
            print("↵") # Símbolo visual para Enter
        elif char_nuevo == ' ':
            print("·", end='') # Puntito para espacio
        else:
            print(char_nuevo, end='') # Caracter normal

        sys.stdout.flush() # Forzar que se imprima ya

        # Parar si termina
        if "<|endoftext|>" in char_nuevo: # A veces se parte en tokens
            break

    print("\n--------------------------------------------------")
    print("🏁 Diagnóstico terminado.")

# PRUEBA ESTO:
# 1. Usamos una temperatura más alta (0.8) para que se arriesgue a pintar algo
# 2. Probamos palabras que SEGURO estaban en el dataset (en inglés)
dibujar_con_rayos_x("dragon", temp=0.8)
dibujar_con_rayos_x("sword", temp=0.8)

🕵️‍♂️ INICIANDO MODO DIAGNÓSTICO (RAYO-X)...

🧪 Pidiendo: 'dragon'
--------------------------------------------------
🧠 Pensando (Caracter a caracter):
·······················.--.··························↵
·······················\·`.························↵
······················|···\······················↵
······················|···\·················
--------------------------------------------------
🏁 Diagnóstico terminado.

🧪 Pidiendo: 'sword'
--------------------------------------------------
🧠 Pensando (Caracter a caracter):
····································______················↵
········___······················|····················↵
·········|·\···············_____|·····················↵
·········|···\············(····
--------------------------------------------------
🏁 Diagnóstico terminado.


In [ ]:
import torch
from torch.nn import functional as F

# Asegúrate de tener 'model', 'stoi', 'itos' y 'device' cargados del paso anterior

def dibujar_agresivo(prompt, penalizacion_espacio=1.2, temp=0.9):
    print(f"\n🎨 Prompt AGRESIVO: '{prompt}'")
    print("--------------------------------------------------")

    start_str = f"<|startoftext|>\nUser: Dibujo ASCII de {prompt}\nAI:\n"
    idx = torch.tensor([[stoi.get(c, 0) for c in start_str]], dtype=torch.long, device=device)

    # Buscamos cuál es el ID del espacio para castigarlo
    id_espacio = stoi.get(' ', -1)

    generated = idx
    conteo_espacios = 0

    # Generamos 1000 tokens
    for _ in range(1000):
        with torch.no_grad():
            logits, _ = model(generated[:, -block_size:])

        logits = logits[:, -1, :]

        # --- AQUÍ ESTÁ EL TRUCO ---
        # Si el token del espacio existe, le restamos probabilidad (logit)
        if id_espacio != -1:
            logits[0, id_espacio] = logits[0, id_espacio] / penalizacion_espacio

        # Aplicamos temperatura alta para que sea creativa
        logits = logits / temp
        probs = F.softmax(logits, dim=-1)

        next_idx = torch.multinomial(probs, num_samples=1)
        generated = torch.cat((generated, next_idx), dim=1)

        # Check fin
        if next_idx.item() == stoi.get('<', 0):
             decoded = ''.join([itos[i] for i in generated[0][-10:].tolist()])
             if "<|endoftext|>" in decoded:
                 break

    # Mostrar resultado
    full_text = ''.join([itos[i] for i in generated[0].tolist()])
    clean_art = full_text.replace(start_str, "").split("<|endoftext|>")[0]
    print(clean_art)
    print("--------------------------------------------------")

# PROBAR CON DISTINTOS NIVELES DE AGRESIÓN
# penalizacion_espacio > 1.0 hace que los espacios sean menos probables
dibujar_agresivo("dragon", penalizacion_espacio=1.5, temp=0.9)
dibujar_agresivo("castle", penalizacion_espacio=2.0, temp=1.0) # Muy agresivo


🎨 Prompt AGRESIVO: 'dragon'
--------------------------------------------------
        ._.  
 . |~~~|_. 
|. |~~.~~\.
|~~|~~.~|~~|
|~~~|~|~|~|~~|
/|~|~|~~|~~|~|
|~~|~|~~|~~|~~~|

--------------------------------------------------

🎨 Prompt AGRESIVO: 'castle'
--------------------------------------------------
,||||||||,>
|||||||||
|||||||||
||||`.`'.`'.`'.`'.|\
|||`'||"`'`'\||.`'`'||
||||'''``.`'`'`'|||
||''`'`'``'---'|||
||''`'`'+`'`'''``'`'`||
||#|/|##/|##|#||#\#|||
||^^^'`'``^'#^`'^||||
||,`^^`'`'`'|||#||#|
|||###^``^^',|||####||
||#\^^^^^^^^'|||####||
||`^^'^^"''`^'|||##|###||
||^^^`^^'^`^^'||||######||
~^^^^^^^^^^^^^^^^^^^^^^^^^^^||

--------------------------------------------------
